# SBM2 configuration tester

This notebook runs the SBM2 detection algorithm with a selected configuration and prints a summary. The summarry includes:
- dates and hours on which the algoritm triggered
    - dates where the algorithm made a correct detection
    - dates where the algorithm made an incorrect detection
    - unknown dates which can be added (counted as incorrect detections by default)
- configuration accuracy
- more?

This notebook can be run and edited via a mybinder link in an online environment. 

Run the cells below and in the ```# Evaluating``` cell, choose which configuration you want to test.

In [1]:
# Import cell
import numpy as np
import pickle

In [2]:
# Functions for getting number of events in rolling buffer (DT_LW)
def AMP_Check(AMP, EPEAK_THRESH): 
    return AMP>EPEAK_THRESH

def EVENTS_Check(NR, NW_THRES): 
    return NR>NW_THRES

def FREQ_Check(Fp, FP_MIN, FP_MAX, RUA): 
    fmin = Fp>(FP_MIN/RUA)
    fmax = Fp<(FP_MAX/RUA)
    temp = fmin.astype(int)+fmax.astype(int)
    temp[temp<2] = 0
    temp[temp==2] = 1
    return temp

def SBM2_Check(AMP, NR, FP, EPEAK_THRESH, NW_THRES, FP_MIN, FP_MAX, RUA):
    A = AMP_Check(AMP, EPEAK_THRESH)
    E = EVENTS_Check(NR, NW_THRES)
    F = FREQ_Check(FP, FP_MIN, FP_MAX, RUA)
    A = A.astype(int)
    E = E.astype(int)
    F = F.astype(int)
    temp = A + E + F
    temp[temp<3] = 0
    temp[temp==3] = 1
    temp = temp.astype(int)
    return temp

def set_conf(conf):
    EPEAK_THRES = conf['EPEAK_THRES']
    FP_MIN = conf['FP_MIN'] 
    FP_MAX = conf['FP_MAX'] 
    NW_THRES = conf['NW_THRES']
    DT_LW = conf['DT_LW']
    N_LW_THRES = conf['N_LW_THRES']
    return EPEAK_THRES, FP_MIN, FP_MAX, NW_THRES, DT_LW, N_LW_THRES

def evaluate(conf):
    EPEAK_THRES, FP_MIN, FP_MAX, NW_THRES, DT_LW, N_LW_THRES = set_conf(conf)
    
    EV = SBM2_Check(AMP, NR, FP, EPEAK_THRES, NW_THRES, FP_MIN, FP_MAX, rua)
    EVi = list(np.where(EV==1))[0]
    
    Ep = Epoch*1e-9
    times = np.array([])
    for i in list(EVi.tolist()):
        E1 = Ep[i]
        temp = Ep[0:i]
        SE = np.sum(EV[np.where(temp>E1-DT_LW)])
        if SE>N_LW_THRES:
            times = np.append(times, time[i])
    
    # Known dates - sbm includes known type III. events; fp includes false positives often returned by SBM2 detection algorithm.
    # Check dates and add them to the correct list if the Evaluation cell printed 'check DATE'. 
    sbm = ['2021/05/22','2021/05/23','2020/12/09','2020/12/10','2020/12/11','2020/12/12','2020/12/13','2020/11/18','2021/05/09','2020/11/17','2021/05/29','2020/12/14']
    fp  = ['2021/03/11','2021/06/01','2021/06/04','2021/06/08','2021/05/10','2021/01/13','2021/01/14','2020/12/06','2020/08/31','2020/08/27','2020/08/29','2020/07/24','2020/04/19','2020/04/18','2020/05/02','2020/07/10','2020/12/17','2021/03/27','2021/03/20','2021/05/31','2021/01/05','2021/03/19','2020/12/27','2020/05/30','2020/07/23','2020/08/04','2020/11/24','2020/12/20','2021/06/07','','','','','','','','','','','','','','']
    
    correct = 0
    incorrect = 0 
    dates = np.array([])
    for i in range(times.size):
        date = times[i].strftime('%Y/%m/%d - %H')
        dates = np.append(dates, date)

    for i in np.unique(dates):
        if i[0:10] in sbm:
            print('correct: '+i)
            correct = correct +1
        else:
            incorrect = incorrect + 1 
            if i[0:10] not in fp:
                print('check ' + i)
            else:
                print('incorrect: ' + i)
            
    print('total correct: ' + str(correct))
    print('total incorrect: ' + str(incorrect))

In [3]:
# Loading data part 1
with open('SBM2_raw_data_V6.pickle', 'rb') as f:
    FP, AMP, NR = pickle.load(f)

In [4]:
# Loading data part 2 (the data loading was split to avoid mybinder timeout errors)
with open('SBM2_timepos_data_V6.pickle', 'rb') as f:
    time, rua, Epoch, et = pickle.load(f)

In [5]:
# CONFIGURATION CELL #
# Confirmation default 
condef = {'EPEAK_THRES': 200, 'FP_MIN': 5000, 'FP_MAX': 50000, 'NW_THRES': 2, 'DT_LW': 304,  'N_LW_THRES': 5 , 'title': '_confirmation', 'type': 'default'}

# Degraded default ('_F' is removed for convenience)
degdef = {'EPEAK_THRES': 728, 'FP_MIN': 5000, 'FP_MAX': 50000, 'NW_THRES': 3, 'DT_LW': 1200, 'N_LW_THRES': 20, 'title': '_degraded', 'type': 'default'}

# Outdated confirmation default 
ocondef = {'EPEAK_THRES': 728, 'FP_MIN': 5000, 'FP_MAX': 60000, 'NW_THRES': 3, 'DT_LW': 2400, 'N_LW_THRES': 20, 'title': 'confirmation', 'type': 'outdated-default'}

# Outdated degraded default ('_F' is removed for convenience)
odegdef = {'EPEAK_THRES': 728, 'FP_MIN': 5000, 'FP_MAX': 60000, 'NW_THRES': 3, 'DT_LW': 1200, 'N_LW_THRES': 20, 'title': 'degraded', 'type': 'outdated-default'}

# Custom config 1
custom1 = {'EPEAK_THRES': 200, 'FP_MIN': 5000, 'FP_MAX': 50000, 'NW_THRES': 1, 'DT_LW': 1200,  'N_LW_THRES': 5 , 'title': 'custom1', 'type': 'dev'}

# Custom config 2
custom2 = {'EPEAK_THRES': 150, 'FP_MIN': 5000, 'FP_MAX': 50000, 'NW_THRES': 1, 'DT_LW': 1200,  'N_LW_THRES': 5 , 'title': 'custom2', 'type': 'dev'}


In [6]:
# Evaluating
evaluate(degdef)

incorrect: 2021/03/11 - 08
incorrect: 2021/03/11 - 09
total correct: 0
total incorrect: 2


In [7]:
evaluate(condef)

correct: 2020/11/17 - 13
correct: 2020/11/17 - 14
incorrect: 2020/12/27 - 04
incorrect: 2020/12/27 - 08
check 2021/01/26 - 13
check 2021/01/26 - 14
check 2021/01/26 - 15
check 2021/01/26 - 16
check 2021/01/26 - 17
check 2021/01/26 - 18
check 2021/02/18 - 14
check 2021/02/18 - 15
check 2021/02/19 - 10
check 2021/02/19 - 11
check 2021/02/19 - 12
check 2021/02/19 - 13
check 2021/02/19 - 20
check 2021/02/22 - 16
check 2021/02/22 - 17
check 2021/02/22 - 18
check 2021/02/22 - 19
check 2021/02/22 - 20
check 2021/03/10 - 21
check 2021/03/10 - 23
incorrect: 2021/03/11 - 00
incorrect: 2021/03/11 - 01
incorrect: 2021/03/11 - 02
incorrect: 2021/03/11 - 03
incorrect: 2021/03/11 - 04
incorrect: 2021/03/11 - 05
incorrect: 2021/03/11 - 06
incorrect: 2021/03/11 - 07
incorrect: 2021/03/11 - 08
incorrect: 2021/03/11 - 09
incorrect: 2021/03/11 - 10
incorrect: 2021/03/11 - 11
incorrect: 2021/03/11 - 12
incorrect: 2021/03/11 - 13
incorrect: 2021/03/11 - 15
incorrect: 2021/03/11 - 16
incorrect: 2021/03/11 - 